In [4]:
###IMPORTS

from __future__ import division

import ladybug.epw as epw

import collections
import array
import math

import rhinoinside
rhinoinside.load()
import Rhino

try:
    import Rhino.UI as rui
except ImportError as e:
    raise ImportError("Failed to import Rhino.\n{}".format(e))

try:
    from ladybug.viewsphere import view_sphere
    from ladybug.graphic import GraphicContainer
    from ladybug.legend import LegendParameters
    from ladybug.color import Colorset
except ImportError as e:
    raise ImportError('\nFailed to import ladybug:\n\t{}'.format(e))

try:
    from ladybug_rhino.config import conversion_to_meters
    from ladybug_rhino.togeometry import to_joined_gridded_mesh3d
    from ladybug_rhino.fromgeometry import from_mesh3d, from_point3d, from_vector3d
    #from ladybug_rhino.fromobjects import legend_objects
    #from ladybug_rhino.text import text_objects
    #from ladybug_rhino.intersect import join_geometry_to_mesh, intersect_mesh_rays
    #from ladybug_rhino.grasshopper import all_required_inputs, hide_output, \
        #show_output, objectify_output, de_objectify_output, recommended_processor_count
except ImportError as e:
    raise ImportError('\nFailed to import ladybug_rhino:\n\t{}'.format(e))


In [5]:
###INCIDENT RADIATION

#Paramètres d'entrée :
_sky_mtx = None
_geometry = None
context_ = None
_grid_size = None
_offset_dist_ = 0.5
irradiance_ = False #set to default : done
legend_par_ = None #set to default
_cpu_count_ = None #set to default

# set the default offset distance and _cpu_count
_offset_dist_ = _offset_dist_ if _offset_dist_ is not None \
    else 0.1 / conversion_to_meters()
workers = 1 # _cpu_count_ if _cpu_count_ is not None else recommended_processor_count()

# create the gridded mesh from the geometry
study_mesh = to_joined_gridded_mesh3d(_geometry, _grid_size)
points = [from_point3d(pt.move(vec * _offset_dist_)) for pt, vec in
          zip(study_mesh.face_centroids, study_mesh.face_normals)]

# mesh the geometry and context
shade_mesh = join_geometry_to_mesh(_geometry + context_)

# deconstruct the matrix and get the sky dome vectors
mtx = de_objectify_output(_sky_mtx)
total_sky_rad = [dir_rad + dif_rad for dir_rad, dif_rad in zip(mtx[1], mtx[2])]
ground_rad = [(sum(total_sky_rad) / len(total_sky_rad)) * mtx[0][1]] * len(total_sky_rad)
all_rad = total_sky_rad + ground_rad 
lb_vecs = view_sphere.tregenza_dome_vectors if len(total_sky_rad) == 145 \
    else view_sphere.reinhart_dome_vectors
if mtx[0][0] != 0:  # there is a north input for sky; rotate vectors
    north_angle = math.radians(mtx[0][0])
    lb_vecs = tuple(vec.rotate_xy(north_angle) for vec in lb_vecs)
lb_grnd_vecs = tuple(vec.reverse() for vec in lb_vecs)
all_vecs = [from_vector3d(vec) for vec in lb_vecs + lb_grnd_vecs]

# intersect the rays with the mesh
normals = [from_vector3d(vec) for vec in study_mesh.face_normals]
int_matrix_init, angles = intersect_mesh_rays(
    shade_mesh, points, all_vecs, normals, cpu_count=workers)

# compute the results
results = []
int_matrix = []
for int_vals, angs in zip(int_matrix_init, angles):
    pt_rel = [ival * math.cos(ang) for ival, ang in zip(int_vals, angs)]
    int_matrix.append(pt_rel)
    rad_result = sum(r * w for r, w in zip(pt_rel, all_rad))
    results.append(rad_result)

# convert to irradiance if requested
if irradiance_:
    factor = 1000 / _sky_mtx.wea_duration if hasattr(_sky_mtx, 'wea_duration') \
        else 1000 / (((mtx[0][3] - mtx[0][2]).total_seconds() / 3600) + 1)
    results = [r * factor for r in results]

# output the intersection matrix and compute total radiation
int_mtx = objectify_output('Geometry/Sky Intersection Matrix', int_matrix)
unit_conv = conversion_to_meters() ** 2
total = 0
for rad, area in zip(results, study_mesh.face_areas):
    total += rad * area * unit_conv


# create the mesh and legend outputs
l_par = legend_par_ if legend_par_ is not None else LegendParameters()
if hasattr(_sky_mtx, 'benefit_matrix') and _sky_mtx.benefit_matrix is not None:
    study_name = '{} Benefit/Harm'.format(study_name)
    if l_par.are_colors_default:
        l_par.colors = reversed(Colorset.benefit_harm())
    if l_par.min is None:
        l_par.min = min((min(results), -max(results)))
    if l_par.max is None:
        l_par.max = max((-min(results), max(results)))
graphic = GraphicContainer(results, study_mesh.min, study_mesh.max, l_par)
graphic.legend_parameters.title = 'kWh/m2' if not irradiance_ else 'W/m2'
title = text_objects(
    study_name, graphic.lower_title_location,
    graphic.legend_parameters.text_height * 1.5,
    graphic.legend_parameters.font)

# create all of the visual outputs
study_mesh.colors = graphic.value_colors
mesh = from_mesh3d(study_mesh)
legend = legend_objects(graphic.legend)

TypeError: 'NoneType' object is not iterable